# Optimizing Model Parameters

In [1]:
!nvidia-smi

Mon Apr 10 00:10:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060         On | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P8               21W / 170W|   1710MiB / 12288MiB |     10%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Prerequisites

In [2]:
# required imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# the training dataset
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

In [4]:
# the test dataset
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
# initialize dataloaders with batch sizes.
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [6]:
# define the network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        return logits

In [7]:
model = NeuralNetwork()

## Hyperparameters for Optimization Tuning

Hyperparameters allow tuning for the model optimization process by affecting model training and convergence.

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

We can each iteration of the optimization loop an **epoch**, which is also one iteration over the entire dataset. Each epoch consists of two main components,
- The *train* loop: iterate over the training dataset and try to converge to optimal parameters.
- The *validation/test* loop: iterate over the test dataset to check if model performance is improving.

### The Loss Function

Here for classification we use the [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss), which combines `LogSoftmax` with `NLLLoss`.



In [9]:
loss_func = nn.CrossEntropyLoss()

### The Optimizer

Here we use the `SGD` optimizer,

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside a training loop, the optimization proceeds in the following three steps,
- Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients add up by default, so to prevent double counting, we explicitly set them to zero at each iteration.
- Backpropagate the prediction loss with `loss.backward()`.
- After gradient computation, call `optimizer.step()` to adjust the parameters based on the backpropagation.

In [11]:
def train_loop(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        # compute prediction and loss
        pred = model(X)
        loss = loss_func(pred, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print loss
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [12]:
def test_loop(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 10

for epoch in range(1, epochs+1):
    print(f"Epoch {epoch}: ")
    train_loop(train_dataloader, model, loss_func, optimizer)
    test_loop(test_dataloader, model, loss_func)

print("Done!")

Epoch 1: 
loss: 2.299685 [    0/60000]
loss: 2.292084 [ 6400/60000]
loss: 2.275720 [12800/60000]
loss: 2.267535 [19200/60000]
loss: 2.251242 [25600/60000]
loss: 2.230010 [32000/60000]
loss: 2.234291 [38400/60000]
loss: 2.203198 [44800/60000]
loss: 2.199979 [51200/60000]
loss: 2.167207 [57600/60000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 2.163139 

Epoch 2: 
loss: 2.176700 [    0/60000]
loss: 2.168253 [ 6400/60000]
loss: 2.112622 [12800/60000]
loss: 2.116625 [19200/60000]
loss: 2.074771 [25600/60000]
loss: 2.022033 [32000/60000]
loss: 2.041827 [38400/60000]
loss: 1.964614 [44800/60000]
loss: 1.971562 [51200/60000]
loss: 1.895895 [57600/60000]
Test Error: 
 Accuracy: 48.5%, Avg loss: 1.893303 

Epoch 3: 
loss: 1.937817 [    0/60000]
loss: 1.905373 [ 6400/60000]
loss: 1.784002 [12800/60000]
loss: 1.805810 [19200/60000]
loss: 1.714355 [25600/60000]
loss: 1.664854 [32000/60000]
loss: 1.683375 [38400/60000]
loss: 1.582166 [44800/60000]
loss: 1.613850 [51200/60000]
loss: 1.510762 [57600/600